Here we will implement the political optimizer algorithm. 
It consists of 4 steps: 

- The main framework (algorithm 1 in the paper)
- The Election Campaign (algorithm 2)
- Party Switching (algorithm 3)
- Parliamentary Affairs (algorithm 4)

For the moment, we can just make this 4 functions in a jupyter notebook. When we have confirmation from the mealpy library, we can start adapting it to the form they want. (they have an optimizer class with some built-in methods etc.)

A first problem is how we want to initialise the population. From the matlab code of the authors, this is how they did it: https://fr.mathworks.com/matlabcentral/fileexchange/74577-political-optimizer-po/
Again, if we can work within the MealPy library, there are built-in functions for this. I will therefore try to make abstraction of this for now. 

In [1]:
import numpy as np
print('test')

In [11]:
def PoliticalOptimizer(fun, n, lambdamax, tmax, d):
    '''
    Parameters: 
    fun -> the function we want to optimize
    n -> the number of contituencies, political parties and party members
    lambdamax -> upper limit of the party switching rate
    tmax -> number of iterations
    
    d -> number of dimensions, we don't want this to be a parameter later I think
    
    Out: the final population
    '''
    
    ################
    # Initialization: sth arbitrary now to replicate the paper, making 9 2D individuals
    # n = 3 parties, d = 2 dimensions
    ################
    
    populationt = np.zeros((n**2, d))
    for i in range(n**2):
        populationt[i,:] = np.random.normal(0,1,2)
    print(populationt) 
    ###############
    # Initializing parties
    # We have n parties and constituencies in the political system, we initialize them here
    # party 1 consists of the first n individuals, party 2 of the following n etc.
    # we store the indices in parties, so that we can always use the indices to refer to individuals in the population
    ###############
    parties = constituencies = np.array([range(n*i, (n*i+n)) for i in range(n)])
    print(f'The parties: {parties}')
    
    ###############
    # Initialize the fitness of all individuals
    ###############
    fitnesst = np.array([fun(x) for x in populationt])    
    print(f'Their fitness: {fitnesst}')
    ################# 
    # Computing the party leaders
    #################
    
    # First we look at the first party, then the second, then the third,...
    
    n_party = 0
    party_leaders = np.zeros(n)
    for party in parties:
        party_leaders[n_party] = n_party*n + np.argmin(fitnesst[party])
        # 0 * 3 + 0,1 or 2. Then 1*3 + 0,1,2
        n_party += 1
        
    print(f'The selected party leaders: {party_leaders}')
    
    ################
    # Computing the constituency leaders, 1 member for each party competes against each other
    # So in our toy example it is individual 0 vs. 3 vs. 6
    # In general it is 0 * n, 1 * n ...
    ################
    
    # first we make the constituencies
    constituencies = []
    
    for const in range(n):
        constituency = []
        for party in parties: 
            constituency.append(party[const])
        constituencies.append(constituency)
    # then we select the leader
    n_constituency = 0
    constituency_leaders = np.zeros(n)
    for constituency in constituencies: 
        # For constituency 0: 0, 3 or 6
        # For constituency 1: 1, 4 or 7
        constituency_leaders[n_constituency] = n_constituency + 3*np.argmin(fitnesst[constituency])
        n_constituency += 1

    print(f'The constituency leaders: {constituency_leaders}')
    
    ################
    # Further initialisations
    ################
    t = 1
    populationtmin1 = populationt
    fitnesstmin1 = fitnesst
    Lambda = lambdamax
    print(f'index of p00 is: {parties[0][0]}')
    print(f'individual p00 is: {populationt[parties[0,0]]}')
    
    ################
    # Starting while loop
    ################
    while(t <= tmax):
        populationtemp = populationt
        fitnesstemp = fitnesst
        for i in range(len(parties)): 
            for j in range(len(party)):
                # partyleaders and constituency leaders give the index of the party leader
                # we can plug this into the population to get the party leader
                populationt[parties[i,j]] = print(ElectionCampaign(fun,d, pji = populationt[parties[i,j]], pjitminus1 = populationtmin1[parties[i,j]], pstari = populationt[int(party_leaders[i])], cstarj = populationt[int(constituency_leaders[j])]))
        parties = PartySwitching(populationt, lambdamax,n)
                  
        t += 1
    
# here we try to minimize the 3D-paraboloïd with center 0        
PoliticalOptimizer(lambda x: x[0]**2 + x[1]**2,3,2,2,2) 

[[ 1.56781866 -1.77325852]
 [ 1.03366785  0.23010261]
 [ 0.21611312  0.88529892]
 [ 0.04650809  1.54436428]
 [-0.24891779  0.24138438]
 [ 0.86587576 -1.61785414]
 [ 0.37784757  0.51774572]
 [-0.82603707  1.2460153 ]
 [-1.04655893 -0.68068525]]
The parties: [[0 1 2]
 [3 4 5]
 [6 7 8]]
Their fitness: [5.60250111 1.12141644 0.83045905 2.38722404 0.12022648 3.36719284
 0.41082941 2.23489137 1.558618  ]
The selected party leaders: [2. 4. 6.]
The constituency leaders: [6. 4. 2.]
index of p00 is: 0
individual p00 is: [ 1.56781866 -1.77325852]
[ 1.13700008 -0.19944379]
[-0.15772991 -0.07287048]
[0.21611312 0.88529892]
[0.73341715 1.3309599 ]
[-0.24891779  0.24138438]
[ 0.86587576 -1.61785414]
[0.37784757 0.51774572]
[-0.82603707  1.2460153 ]
[-1.04655893 -0.68068525]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]
[nan nan]


In [10]:
# TODO: Check that everything is right, but it runs

def ElectionCampaign(fun, d, pji, pjitminus1, pstari, cstarj):
    if(fun(pji) <= fun(pjitminus1)):
        for k in range(d):
            mstar = pstari[k]
            r = np.random.rand()
            # Implementation of eq9
            if((pjitminus1[k] <= pji[k] <= mstar) or (pjitminus1[k] >= pji[k] >= mstar)):
                pji[k] = mstar + r*(mstar - pji[k])
            elif((pjitminus1[k] <= mstar <= pji[k]) or (pjitminus1[k] >= mstar >= pji[k])):
                pji[k] = mstar = (2*r-1)*abs(mstar - pji[k])
            elif((mstar <= pjitminus1[k] <= pji[k]) or (mstar >= pjitminus1[k] >= pji[k])):
                pji[k] = mstar + (2*r-1)* abs(mstar - pjitminus1[k])
            mstar = cstarj[k]
            r = np.random.rand()
            # This is just a copy paste of what was above, since eq 9 already implemented
            if((pjitminus1[k] <= pji[k] <= mstar) or (pjitminus1[k] >= pji[k] >= mstar)):
                pji[k] = mstar + r*(mstar - pji[k])
            elif((pjitminus1[k] <= mstar <= pji[k]) or (pjitminus1[k] >= mstar >= pji[k])):
                pji[k] = mstar = (2*r-1)*abs(mstar - pji[k])
            elif((mstar <= pjitminus1[k] <= pji[k]) or (mstar >= pjitminus1[k] >= pji[k])):
                pji[k] = mstar + (2*r-1)* abs(mstar - pjitminus1[k])
    else:
        for k in range(d):
            mstar = pstari[k]
            r = np.random.rand()
            # Implementation of eq 10
            if((pjitminus1[k] <= pji[k] <= mstar) or (pjitminus1[k] >= pji[k] >= mstar)):
                pji[k] = mstar + (2*r-1)*abs(mstar - pji[k])
            elif((pjitminus1[k] <= mstar <= pji[k]) or (pjitminus1[k] >= mstar >= pji[k])):
                pji[k] = pjitminus1[k] + r*(pji[k] - pjitminus1[k])
            elif((mstar <= pjitminus1[k] <= pji[k]) or (mstar >= pjitminus1[k] >= pji[k])):
                pji[k] = mstar + (2*r - 1) * abs(mstar - pjitminus1[k])
            mstar = cstarj[k]
            r = np.random.rand()
            # Copy paste of what was above, since eq 10 already implemented
            if((pjitminus1[k] <= pji[k] <= mstar) or (pjitminus1[k] >= pji[k] >= mstar)):
                pji[k] = mstar + (2*r-1)*abs(mstar - pji[k])
            elif((pjitminus1[k] <= mstar <= pji[k]) or (pjitminus1[k] >= mstar >= pji[k])):
                pji[k] = pjitminus1[k] + r*(pji[k] - pjitminus1[k])
            elif((mstar <= pjitminus1[k] <= pji[k]) or (mstar >= pjitminus1[k] >= pji[k])):
                pji[k] = mstar + (2*r - 1) * abs(mstar - pjitminus1[k])
    return pji

In [ ]:
def PartySwitching(population, Lambda,n):
    for i in range(len(parties)): 
            for j in range(len(party)):
                sp = np.random.rand()
                if(sp < Lambda):
                    r = np.random.randint(0,n)
                    # Determine q with eq11
                    # swap pqr and pji
    return 0

In [ ]:
def ParliamentaryAffairs(constituencyLeaders, population,n):
    for j in range(n):
        r = np.random.randint(0,n)
        a = np.random.rand(0)
        
    return 0